<a href="https://colab.research.google.com/github/sowmya-520/NNDL/blob/main/NNDL_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2

Estimate load at particular hour on 33/11KV substation based on load at same time but previous day. Consider momentum+SGD optimizer.
Dataset: Active power load dataset


• Write the python code to build simple linear regression model using momentum+SGD optimizer.
• Do the data normalization
• Split the data for train and test (90:10)
• Train the simple linear regression model training data
• Compute MSE, RMSE and MAE with training data
• Compute MSE, RMSE and MAE with testing data

#Mount Google Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import libraries

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Load the data

In [20]:
data = pd.read_excel('/content/drive/MyDrive/Data2.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [21]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])

In [22]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()


,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


#Data Normalisation

In [23]:
maxX = np.max(data['Previous Day Load'])
minX = np.min(data['Previous Day Load'])
maxY = np.max(data['Present Day Load'])
minY = np.min(data['Present Day Load'])

data['Previous Day Load'] = (data['Previous Day Load'] - minX) / (maxX - minX)
data['Present Day Load'] = (data['Present Day Load'] - minY) / (maxY - minY)

#Split the Data

In [24]:
x = data['Previous Day Load']
y = data['Present Day Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

#momentum + SGD optimiser algorithm

In [25]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5
for _ in range(epochs):
    for i in range(trainX.shape[0]):
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        m += vm
        c += vc

print(f'm = {m}\nc = {c}')

m = 0.7171132537325571
c = 0.12511078784145233


#Model Prediction And Error metrics calculation

In [26]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 533.2583331279446
MSE : 519633.2481810661
RMSE : 720.8559136062256
Testing Error :
MAE : 546.2799015481903
MSE : 497098.195375769
RMSE : 705.0519097029445


In [27]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,4221.85392,4606.326987
1,7609.43808,6503.071349
2,5847.05880,5918.219480
3,4672.50300,4994.327911
4,7883.78688,6613.066425
...,...,...
1939,5830.37964,5678.778979
1940,6110.49600,5870.823228
1941,5289.94368,5445.151226
1942,7962.97392,7536.399077


In [28]:
x = float(input('Enter the load at present hour on previouus day : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previouus day : 4221
Predicted load at present hour : 4666.077780857587
